In [65]:
import pandas as pd
import seaborn as sns
import scipy.stats as st
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [66]:
#Extract Combine data from pro-football reference for years 2009 to 2020
years = ['2009','2010','2011','2012','2013','2014','2015','2016','2017','2018','2019','2020']
dfs=[]
for yr in years:
    #Adjust URL based on years fed in
    url = f"https://www.pro-football-reference.com/play-index/nfl-combine-results.cgi?request=1&year_min={yr}&year_max={yr}&height_min=60&height_max=82&weight_min=120&weight_max=400&pos%5B%5D=WR&show=all&order_by=year_id"
    try:
        #read table on page into a dataframe
        df = pd.read_html(url)
        #drop AV column
        new_df = df[0].drop(columns="AV")
        
        #save dataframe for potential reference later
        dfs.append(new_df)
        
        #Output success message
        print(f"success: {yr}")
        
        #Create first dataframe and then add subsequent dataframes
        if len(dfs)==1:
            comb_df = new_df
        else:
            comb_df = pd.concat([comb_df,new_df])
                
    except: 
        #Notification of years that couldn't be read in
        print(yr)

success: 2009
success: 2010
success: 2011
success: 2012
success: 2013
success: 2014
success: 2015
success: 2016
success: 2017
success: 2018
success: 2019
success: 2020


In [67]:
#Split out draft info 
comb_df[["Team","Round","pickNum","year"]] = comb_df["Drafted (tm/rnd/yr)"].str.split("/",n=3,expand=True)

#Extract just the number for pickNum and Round
comb_df["pickNum"] = comb_df["pickNum"].str.extract('(\d+)')
comb_df["Round"] = comb_df["Round"].str.extract('(\d+)')

#Get rid of combine results of those not drafted
comb_df["pickNum"].fillna("Not drafted",inplace = True)
drafteddf = comb_df[comb_df["pickNum"] != "Not drafted"]

#Remove header columns
drafteddf=drafteddf[drafteddf["Year"] != "Year" ]


#Remove columns that were split out and College stats link column
combine= drafteddf.drop(columns = ['Drafted (tm/rnd/yr)','College'])


#Convert Height in Feet and inches to inches
combine[['hfeet','hinches']] = combine['Height'].str.split("-",n=1,expand=True)
combine[['hfeet','hinches']] = combine[['hfeet','hinches']].astype(int)
combine['height(in)']=12*combine['hfeet']+combine['hinches']
combine.drop(columns = ['Height','hfeet','hinches'], inplace = True)

#Separate out players first and last name for joining later
combine[["FirstNm","Last"]] = combine["Player"].str.split(" ",n=1,expand=True)
combine['FirstNm']=combine['FirstNm'].str.upper()
combine['Last']=combine['Last'].str.upper()

#Get rid of , and . which may be inconsistent based on sample
combine['Last']=combine['Last'].str.replace('.','')
combine['Last']=combine['Last'].str.replace(',','')
combine['FirstNm']=combine['FirstNm'].str.replace('.','')
combine['FirstNm']=combine['FirstNm'].str.replace(',','')

#Add preceeding , to common suffixes for parsing out
combine['Last']=combine['Last'].str.replace('JR',',JR')
combine['Last']=combine['Last'].str.replace('II',',II')
combine['Last']=combine['Last'].str.replace('III',',III')
combine['Last']=combine['Last'].str.replace('IV',',IV')

#parse out the suffix from the last name
combine[['LastNm','Suffix']]=college_career_df['Last'].str.split(",",n=1,expand=True)
combine

,Rk,Year,Player,Pos,Age,School,Wt,40YD,Vertical,BenchReps,Broad Jump,3Cone,Shuttle,Team,Round,pickNum,year,height(in),FirstNm,LastNm
0,1,2009,Derrick Williams,WR,22,Penn State,194,4.47,33,15,NaN,NaN,NaN,Detroit Lions,3,82,2009,71,DERRICK,WILLIAMS
1,2,2009,Mike Wallace,WR,22,Mississippi,199,4.33,40,14,129,6.9,4.27,Pittsburgh Steelers,3,84,2009,72,MIKE,WALLACE
2,3,2009,Tiquan Underwood,WR,22,Rutgers,184,4.31,41.5,7,129,6.62,4.11,Jacksonville Jaguars,7,253,2009,73,TIQUAN,UNDERWOOD
3,4,2009,Patrick Turner,WR,21,USC,223,4.59,NaN,NaN,NaN,NaN,NaN,Miami Dolphins,3,87,2009,77,PATRICK,TURNER
4,5,2009,Mike Thomas,WR,21,Arizona,195,4.3,40.5,NaN,126,6.65,4.28,Jacksonville Jaguars,4,107,2009,68,MIKE,THOMAS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46,47,2020,Tyrie Cleveland,WR,NaN,Florida,209,4.46,39.5,13,126,NaN,NaN,Denver Broncos,7,252,2020,74,TYRIE,CLEVELAND
47,48,2020,Chase Claypool,WR,21,Notre Dame,238,4.42,40.5,19,126,NaN,NaN,Pittsburgh Steelers,2,49,2020,76,CHASE,CLAYPOOL
48,49,2020,Quintez Cephus,WR,NaN,Wisconsin,202,4.73,38.5,23,124,7.20,4.33,Detroit Lions,5,166,2020,73,QUINTEZ,CEPHUS
53,53,2020,Lynn Bowden,WR,22,Kentucky,204,NaN,NaN,13,NaN,NaN,NaN,Las Vegas Raiders,3,80,2020,71,LYNN,BOWDEN


In [109]:
combine.to_csv('combine.csv')

In [115]:
givens = combine[combine['Player']=='Jalen Hurd']
givens

,Rk,Year,Player,Pos,Age,School,Wt,40YD,Vertical,BenchReps,Broad Jump,3Cone,Shuttle,Team,Round,pickNum,year,height(in),FirstNm,LastNm
22,23,2019,Jalen Hurd,WR,23,Baylor,226,NaN,NaN,23,NaN,NaN,NaN,San Francisco 49ers,3,67,2019,77,JALEN,HURD


In [88]:
trial = combine.copy()

In [68]:


def splitsuffix (lastname):
    if ('JR' in lastname or 'III' in lastname or 'II' in lastname):
        return lastname.split(" ")
    
    else:
        return [lastname,""]

In [94]:
trial

,Rk,Year,Player,Pos,Age,School,Wt,40YD,Vertical,BenchReps,...,Shuttle,Team,Round,pickNum,year,height(in),FirstNm,LastNm,suffix,Last
0,1,2009,Derrick Williams,WR,22,Penn State,194,4.47,33,15,...,NaN,Detroit Lions,3,82,2009,71,DERRICK,WILLIAMS,,WILLIAMS
1,2,2009,Mike Wallace,WR,22,Mississippi,199,4.33,40,14,...,4.27,Pittsburgh Steelers,3,84,2009,72,MIKE,WALLACE,,WALLACE
2,3,2009,Tiquan Underwood,WR,22,Rutgers,184,4.31,41.5,7,...,4.11,Jacksonville Jaguars,7,253,2009,73,TIQUAN,UNDERWOOD,,UNDERWOOD
3,4,2009,Patrick Turner,WR,21,USC,223,4.59,NaN,NaN,...,NaN,Miami Dolphins,3,87,2009,77,PATRICK,TURNER,,TURNER
4,5,2009,Mike Thomas,WR,21,Arizona,195,4.3,40.5,NaN,...,4.28,Jacksonville Jaguars,4,107,2009,68,MIKE,THOMAS,,THOMAS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46,47,2020,Tyrie Cleveland,WR,NaN,Florida,209,4.46,39.5,13,...,NaN,Denver Broncos,7,252,2020,74,TYRIE,CLEVELAND,,GILYARD
47,48,2020,Chase Claypool,WR,21,Notre Dame,238,4.42,40.5,19,...,NaN,Pittsburgh Steelers,2,49,2020,76,CHASE,CLAYPOOL,,GETTIS
48,49,2020,Quintez Cephus,WR,NaN,Wisconsin,202,4.73,38.5,23,...,4.33,Detroit Lions,5,166,2020,73,QUINTEZ,CEPHUS,,FORD
53,53,2020,Lynn Bowden,WR,22,Kentucky,204,NaN,NaN,13,...,NaN,Las Vegas Raiders,3,80,2020,71,LYNN,BOWDEN,,BRYANT


In [79]:
trial_df = combine['LastNm'].apply(splitsuffix)

In [80]:
trial_df = trial_df.reset_index()

In [93]:
trial['Last'] = trial_df['LastNm'].str[0]
trial['suffix'] = trial_df['LastNm'].str[1]

In [96]:
trial_df['Last_Name']=trial['LastNm'].str()

TypeError: 'StringMethods' object is not callable

In [48]:
combine.to_csv("evidence.csv")

In [105]:
trial_df = combine.apply(splitif(combine,'LastNm')

In [112]:

college_df = pd.read_csv("college.csv")
finaldf = combine.merge(college_df, how = 'left', on = 'Player')
finaldf.to_csv("final.csv")

In [114]:
wonulls = finaldf.dropna()

In [29]:
#Check how many missing records there are for categorical data
categdata = ['Rk', 'Year', 'Pos', 'Age', 'School','Height', 'Wt','Team',
       'Round', 'pickNum', 'year', 'First', 'Last']
for i in categdata:
    try:
        nanCount = combine.groupby(i)['40YD'].count().loc['NoRecord']
        print(f"{i} is missing {nanCount} records")
    except:
        print(f"{i} has all records filled")

In [31]:
drafteddf.to_csv("combine_drafted")

In [115]:
wonulls

,Rk,Year,Player,Pos,Age,School,Wt,40YD,Vertical,BenchReps,...,height(in),Rec,Yds,TD,Att,Yds.1,TD.1,Plays,Yds.2,TD.2
0,1,2009,Derrick Williams,WR,22,Penn State,194,4.47,33,15,...,71,139.0,1454.0,8.0,95.0,489.0,5.0,234.0,1943.0,13.0
1,2,2009,Mike Wallace,WR,22,Mississippi,199,4.33,40,14,...,72,101.0,1910.0,15.0,11.0,94.0,0.0,112.0,2004.0,15.0
2,3,2009,Tiquan Underwood,WR,22,Rutgers,184,4.31,41.5,7,...,73,105.0,1594.0,12.0,0.0,0.0,0.0,105.0,1594.0,12.0
3,4,2009,Patrick Turner,WR,21,USC,223,4.59,35.907,14.9605,...,77,126.0,1582.0,15.0,0.0,0.0,0.0,126.0,1582.0,15.0
4,5,2009,Mike Thomas,WR,21,Arizona,195,4.3,40.5,14.9605,...,68,259.0,3231.0,22.0,45.0,395.0,3.0,304.0,3626.0,25.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329,45,2020,Gabriel Davis,WR,NoRecord,UCF,216,4.54,35.0,14,...,74,152.0,2447.0,23.0,0.0,0.0,0.0,152.0,2447.0,23.0
331,47,2020,Tyrie Cleveland,WR,NoRecord,Florida,209,4.46,39.5,13,...,74,47.0,761.0,3.0,2.0,6.0,0.0,49.0,767.0,3.0
332,48,2020,Chase Claypool,WR,21,Notre Dame,238,4.42,40.5,19,...,76,145.0,2078.0,19.0,2.0,-3.0,0.0,147.0,2075.0,19.0
333,49,2020,Quintez Cephus,WR,NoRecord,Wisconsin,202,4.73,38.5,23,...,73,89.0,1402.0,13.0,4.0,5.0,0.0,93.0,1407.0,13.0


In [ ]:
#Function to replace null values with an average factored by a penalty
def fill_null_floats(df,col_headers,penalty=0):
    df = df.fillna("NoRecord")
    for col in col_headers:
        mean_df = df[df[col]!='NoRecord']
        average = mean_df[col].astype(float).mean()
        print(average)
        df[col]=df[col].replace(to_replace='NoRecord',value = (average-penalty))
    return df

#list of columns that need to be floats and adjusted to null values
col_replace = ['40YD','Vertical', 'BenchReps', 'Broad Jump', '3Cone', 'Shuttle']

#replace null values and convert measurables to floats using function defined above
combine = fill_null_floats(drafteddf,col_replace)